In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tkinter as tk
from PIL import Image, ImageTk
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

# Loading pre-trained emotion recognition model
emotion_model = tf.keras.models.load_model('emotion_detection_model.h5')

# Defining emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Initialize the video capture
cap = cv2.VideoCapture(0)

# Flag to control emotion detection
emotion_detection_active = False

def start_emotion_detection():
    global emotion_detection_active
    emotion_detection_active = True
    button_start.config(state=tk.DISABLED)
    button_pause.config(state=tk.NORMAL)

def pause_emotion_detection():
    global emotion_detection_active
    emotion_detection_active = False
    button_start.config(state=tk.NORMAL)
    button_pause.config(state=tk.DISABLED)

def exit_emotion_detection():
    root.destroy()

# tkinter window
root = tk.Tk()
root.title("Emotion Detection")
root.geometry("650x400")  # Set the window size

# label to display the video feed
label = tk.Label(root)
label.grid(row=0, column=0, columnspan=3, padx=10, pady=10)

# buttons for controlling emotion detection
button_start = tk.Button(root, text="Start Emotion Detection", command=start_emotion_detection)
button_start.grid(row=1, column=0, padx=10, pady=10)

button_pause = tk.Button(root, text="Pause Emotion Detection", command=pause_emotion_detection, state=tk.DISABLED)
button_pause.grid(row=1, column=1, padx=10, pady=10)

button_exit = tk.Button(root, text="Exit Emotion Detection", command=exit_emotion_detection)
button_exit.grid(row=1, column=2, padx=10, pady=10)

def update_frame():
    if emotion_detection_active:
        # Read a frame from the video stream
        ret, frame = cap.read()

        if not ret:
            print("Failed to capture frame")
            root.destroy()

        # Resize the frame for the emotion model input
        input_frame = cv2.resize(frame, (48, 48))
        input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        input_frame = np.expand_dims(input_frame, axis=0)
        input_frame = np.expand_dims(input_frame, axis=-1)

        # Normalize pixel values to be between 0 and 1
        input_frame = input_frame / 255.0

        # Predict emotion for the frame
        emotion_probabilities = emotion_model.predict(input_frame)
        predicted_emotion = emotion_labels[np.argmax(emotion_probabilities)]

        # Display the emotion prediction on the frame
        cv2.putText(frame, f'Emotion: {predicted_emotion}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Convert the frame to RGB format for displaying in tkinter
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(img)

        # Update the label with the new image
        label.img = img
        label.config(image=img)

    # Schedule the update after 10 milliseconds
    root.after(10, update_frame)

# Schedule the initial update
update_frame()

# Run the tkinter event loop
root.mainloop()

# Release the video capture object
cap.release()
